In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import kuaizi as kz
from astropy.table import Table
import astropy.units as u

from kuaizi.display import display_single
from astropy.io import fits
import os
import sys
from shutil import copyfile

In [4]:
kz.utils.set_env(project='HSC', name='LSBG', data_dir='/scratch/gpfs/jiaxuanl/Data/')

'/scratch/gpfs/jiaxuanl/Data/HSC/LSBG'

In [20]:
cat = Table.read('./Cutout/NSA/nsa_cutout_cat_1013.fits')

In [21]:
flag = (cat['bad_votes'] >= cat['good_votes'])

In [22]:
cat = cat[flag] # junks only
len(cat)

253

In [52]:
cat.sort('viz-id')

In [54]:
cat[20:30]

ra,dec,a_image,b_image,theta_image,ellipticity,mag_auto_g,mag_auto_r,mag_auto_i,flux_radius_ave_g,flux_radius_ave_i,mu_ave_g,mu_ave_i,acorr_ratio,g-i,g-r,A_g,A_r,A_i,tract,patch,synth_id,viz-id,is_candy,is_galaxy,is_tidal,is_outskirts,is_cirrus,is_junk,num_votes,good_votes,bad_votes,cutout_size,prefix,radius,image_flag [5],psf_flag [5]
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,arcmin,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,bytes3,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,bytes57,float64,bool,bool
179.9431534,-0.3075206,22.721,8.225,-45.89,0.638,20.8818,17.9939,17.8252,4.0971,6.253548,25.28073470360712,23.71273970899086,1.52997911132378,0.9311743153421118,0.6721127691190681,0.07855033698307193,0.055663106102139745,0.039724652325184844,9347,"0,6",-1,5005,0,1,0,0,0,2,3,1,2,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_5005,1.0,True .. True,True .. True
29.9535664,-0.6277608,13.629,10.88,-2.55,0.202,21.7978,21.6723,21.8402,3.3796560000000007,2.952516,25.983881322827234,25.918388227663055,1.1779059776061858,0.15557461548759752,0.18663297108217528,0.07470573725194335,0.0529387083341176,0.037780352739538316,9246,"2,5",-1,5593,1,0,0,0,0,2,3,1,2,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_5593,1.0,True .. True,True .. True
28.8982299,-3.2208263,5.809,4.055,-32.52,0.302,22.7782,22.4409,23.2093,3.5605920000000006,2.1408240000000003,27.091962905614455,26.350559205668986,1.6839071486977562,-0.060534529242720854,0.30612970804340306,0.07128487674701087,0.050514584790411966,0.03605034750428789,8761,"1,7",-1,5764,1,0,0,0,0,2,3,1,2,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_5764,1.0,True .. True,True .. True
174.7703173,-0.0486206,10.139,9.439,36.08,0.069,20.9842,20.2156,19.9038,2.7557039999999997,2.7016080000000002,24.490320212289177,23.379576575150896,1.4290916367857363,1.0546012747406661,0.7416297877382415,0.06716613189200936,0.04759591963024851,0.03396740663267408,9587,"4,0",-1,5798,0,1,1,0,0,1,3,1,2,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_5798,1.0,True .. True,True .. True
191.3209885,-0.4156015,52.857,26.794,83.77,0.493,19.2198,18.2435,18.0943,13.179264000000002,16.491636,26.175655660207717,25.83670232181116,1.0925591556692758,0.692914749285577,0.4828250900043876,0.06855507857304173,0.048580168577432296,0.03466982785862148,9355,"3,6",-1,5800,1,0,0,2,0,0,3,1,2,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_5800,1.0,True .. True,True .. True
220.1851672,-0.2810931,25.787,11.213,1.96,0.565,20.7246,19.7064,19.3156,5.475456,5.348448,25.698655521128504,24.314719198307433,1.2234096823108611,1.3753813511102433,0.9976977676757519,0.10949035786019166,0.07758812553594156,0.05537170897043407,9375,"7,7",-1,5839,0,1,0,2,0,0,3,1,2,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_5839,1.0,True .. True,True .. True
149.4235478,1.6963401,10.947,6.909,-46.94,0.369,21.808,20.9488,20.4106,2.6656560000000002,1.992396,25.371178872200815,23.295480630028162,1.3373185473733842,1.3745439012713108,0.7435791958683211,0.056357175963612546,0.039936371831932056,0.02850107723492314,9813,"8,1",-1,6007,0,1,0,0,0,2,3,1,2,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_6007,1.0,True .. True,True .. True
34.7252375,-6.6224162,8.111,7.92,88.09,0.024,22.2129,20.9467,20.4588,2.740164,6.627348,25.896461866508307,26.26122780695144,1.3669271800731604,1.034185257653118,0.43987078392317647,0.08075366834009606,0.057224452263272524,0.04083892599321282,8282,"2,4",-1,6321,0,1,0,0,0,2,3,1,2,1.0,/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/nsa_6321,1.0,True .. True,True .. True


### Generate RGB images

In [23]:
def _display_cutout_rgb(images, ax=None):
    import scarlet
    from scarlet.display import AsinhMapping

    if ax is None:
        fig, ax = plt.subplots(figsize=(5, 5))
        
    # Display
    norm = AsinhMapping(minimum=-0.2, stretch=1, Q=1)
    img_rgb = scarlet.display.img_to_rgb(images, norm=norm)
    plt.imshow(img_rgb, origin='lower')
    ax.axis('off')
    
    if ax is None:
        return fig
    return ax

In [31]:
CUTOUT_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/'
FIGURE_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/'
WEBPAGE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/figure/junk/'

In [25]:
# Check file exsistence
for filt in 'grizy':
    for obj in cat:
        if not os.path.isfile(CUTOUT_DIR + f"nsa_{obj['viz-id']}_{filt}.fits"):
            print(f"NSA/nsa_{obj['viz-id']}_{filt}.fits not found!")

NSA/nsa_32934_g.fits not found!
NSA/nsa_32934_r.fits not found!
NSA/nsa_23610_i.fits not found!
NSA/nsa_24979_i.fits not found!
NSA/nsa_32934_i.fits not found!
NSA/nsa_23610_y.fits not found!
NSA/nsa_24979_y.fits not found!
NSA/nsa_32934_y.fits not found!


In [26]:
# Display each cutout
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try:
        images = np.asarray([fits.open(f'{CUTOUT_DIR}/nsa_{ind}_{filt}.fits')[1].data for filt in 'grizy'])
        fig, ax = plt.subplots(figsize=(5, 5))
        ax = _display_cutout_rgb(images, ax=ax)
        plt.savefig(FIGURE_DIR + f'nsa_{ind}_cutout.png', dpi=70, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(ind, e)
        
    if i % 10 == 0:
        print(f'Progress: {i} / {len(cat)}')

Progress: 0 / 253
Progress: 10 / 253
Progress: 20 / 253
Progress: 30 / 253
Progress: 40 / 253
Progress: 50 / 253
Progress: 60 / 253
Progress: 70 / 253
Progress: 80 / 253
Progress: 90 / 253
23610 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA//nsa_23610_i.fits'
Progress: 100 / 253
24979 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA//nsa_24979_i.fits'
Progress: 110 / 253
Progress: 120 / 253
Progress: 130 / 253
32934 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA//nsa_32934_g.fits'
Progress: 140 / 253
Progress: 150 / 253
Progress: 160 / 253
Progress: 170 / 253
Progress: 180 / 253
Progress: 190 / 253
Progress: 200 / 253
Progress: 210 / 253
Progress: 220 / 253
Progress: 230 / 253
Progress: 240 / 253
Progress: 250 / 253


In [33]:
# Copy figures in FIGURE_DIR to WEBPAGE_DIR
for i, obj in enumerate(cat):
    ind = obj['viz-id']
    try:
        if not os.path.isdir(WEBPAGE_DIR):
            os.mkdir(WEBPAGE_DIR)
            
        copyfile(FIGURE_DIR + f'nsa_{ind}_cutout.png', WEBPAGE_DIR + f'nsa_{ind}_cutout.png')
    except Exception as e:
        print(ind, e)

23610 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_23610_cutout.png'
24979 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_24979_cutout.png'
32934 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_32934_cutout.png'


### Generate webpage for cutouts. WE use `diezi/webpage/gen_web_cutout.py`

In [34]:
sys.path.append('/home/jiaxuanl/Research/Packages/kuaizi/diezi/webpage/')
from gen_web_cutout import webpage_cutout

In [48]:
#FOR NSA SAMPLE
FIGURE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/figure/junk'
WEBPAGE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/junk'
SCARLET_DIR = '/tigress/jiaxuanl/public_html/NSA/scarlet_result/'
sample_name = 'NSA'

In [49]:
webpage_cutout(FIGURE_DIR, WEBPAGE_DIR, sample_name, sample_title='Junk', SCARLET_DIR=SCARLET_DIR, 
               col_num=6, row_num=4)

You have 250 galaxies to be displayed
Total pages: 11


In [50]:
print('The webpages are in', WEBPAGE_DIR)

The webpages are in /tigress/jiaxuanl/public_html/NSA/cutout_RGB/junk


### The search box on webpage still doesn't work. Need to work on the javascript file